In [1]:
import re
import string
import pythainlp
from pythainlp import word_tokenize
from pythainlp.corpus import wordnet
from nltk.stem.porter import PorterStemmer
from nltk.corpus import words
import nltk
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import xlrd  
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

from sklearn import preprocessing
from sklearn.model_selection import ShuffleSplit

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc

import sklearn.ensemble as ensemble
import numpy
from imblearn.ensemble import BalancedBaggingClassifier

Using TensorFlow backend.


In [2]:
p_stemmer = PorterStemmer()
# text preprocession
def split_word(text):
    
    tokens = word_tokenize(text,engine='newmm')

    # หารากศัพท์ภาษาไทย และภาษาอังกฤษ
    # English
    tokens = [p_stemmer.stem(i) for i in tokens]
    
    # Thai
    tokens_temp=[]
    for i in tokens:
        w_syn = wordnet.synsets(i)
        if (len(w_syn)>0) and (len(w_syn[0].lemma_names('tha'))>0):
            tokens_temp.append(w_syn[0].lemma_names('tha')[0])
        else:
            tokens_temp.append(i)
    
    tokens = tokens_temp
    
    # ลบตัวเลข
    tokens = [i for i in tokens if not i.isnumeric()]
    
    # ลบช่องว่าง
    tokens = [i for i in tokens if not ' ' in i]

    return tokens

def createGeneralEvaluation(file):
    f = open(file, mode='w')
    f.write("time,amount_of_data,data_type,threshold,type_class,accuracy,precision,recall,fscore,support,auc_roc,auc_pr")
    f.write("\n")
    f.close()
    

def report_performance(file,time,modelType,predictions_class_train_c, predictions_class_test_c, type_class,threshold, predict_train_prob,predict_test_prob, train_label,train_feat):
    #Evaluation class (test data)
    print("Test data of "+modelType)
    data_type_test = "Test data of "+modelType
    threshold = str(threshold)
    type_class = str(type_class)
    accuracy_test = str(accuracy_score(predictions_class_test_c, Test_Y[type_class]))
    print(type_class+" Accuracy score:", 
          accuracy_score(predictions_class_test_c, Test_Y[type_class])*100)
    print("Confusion Matrix: \n", confusion_matrix(Test_Y[type_class],predictions_class_test_c))
    precision,recall,fscore,support=score(Test_Y[type_class],predictions_class_test_c)
    precision_test = (format(precision[1]))
    recall_test = (format(recall[1]))
    fscore_test = (format(fscore[1]))
    support_test = (format(support[1]))
    auc_roc_test = str(roc_auc_score(Test_Y[type_class], predict_test_prob))
    print("PRF: \n", classification_report(Test_Y[type_class],predictions_class_test_c))
    print("Area under the curve ROC: ", roc_auc_score(Test_Y[type_class], predict_test_prob))
    lr_precision_test, lr_recall_test, _ = precision_recall_curve(Test_Y[type_class], predict_test_prob)
    auc_pr_test = str(auc(lr_recall_test, lr_precision_test))
    print("Area under the curve PR: ", auc(lr_recall_test, lr_precision_test))
    print("-----------------------------------------------------------\n")

    #Evaluation class (train data)
    print("Train data of "+modelType)
    data_type_train = "Train data of "+modelType
    accuracy_train = str(accuracy_score(predictions_class_train_c, train_label))
    print(type_class+" Accuracy score:", 
          accuracy_score(predictions_class_train_c, train_label)*100)
    print("Confusion Matrix: \n",confusion_matrix(train_label,predictions_class_train_c))
    precision,recall,fscore,support=score(train_label,predictions_class_train_c)
    precision_train = (format(precision[1]))
    recall_train  = (format(recall[1]))
    fscore_train  = (format(fscore[1]))
    support_train  = (format(support[1]))
    auc_roc_train  = str(roc_auc_score(train_label, predict_train_prob))
    print("PRF: \n",classification_report(train_label,predictions_class_train_c))
    print("Area under the curve ROC: ", roc_auc_score(train_label, predict_train_prob))
    lr_precision_train, lr_recall_train, _ = precision_recall_curve(train_label, predict_train_prob)
    auc_pr_train  = str(auc(lr_recall_train, lr_precision_train))
    print("Area under the curve PR: ", auc(lr_recall_train, lr_precision_train))
    print("___________________________________________________________________________________________________________\n")
    
    f = open(file, mode='a')
    f.write(str(time)+","+str(train_feat)+","+data_type_test+","+threshold+","+type_class+","+accuracy_test+","+precision_test+","+recall_test+","+fscore_test+","+support_test+","+auc_roc_test+","+auc_pr_test)
    f.write("\n")
    f.write(str(time)+","+str(train_feat)+","+data_type_train+","+threshold+","+type_class+","+accuracy_train+","+precision_train+","+recall_train+","+fscore_train+","+support_train+","+auc_roc_train+","+auc_pr_train)
    f.write("\n")
    
    f.close()

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import tree
def train_and_label(train_X, train_y, test_X, test_y, file, type_class, time, dataType,classifier):
    if(classifier =='randomforest'):
        model = RandomForestClassifier(n_estimators=20, n_jobs=20)
    elif(classifier =='logisticRegression'):
        model = LogisticRegression()
    elif(classifier == 'knn'):
        model = KNeighborsClassifier(n_neighbors=3)
    elif(classifier == 'tree'):
        model = tree.DecisionTreeClassifier()
    elif(classifier == 'mlp'):
        model = MLPClassifier()
    elif(classifier == 'svm'):
        model = LinearSVC()
        
    model.fit(train_X, train_y)
    
    # eval
    if(classifier != 'svm'):
        predictions_test = model.predict_proba(test_X)
        predictions_train = model.predict_proba(train_X)

    #only for svc
    else:
        predictions_test = model._predict_proba_lr(test_X)
        predictions_train = model._predict_proba_lr(train_X)
        
    for th in numpy.arange(0.1,1.1,0.1):
        predictions_class_Train = (predictions_train[:,1] >= th).astype('int')
        predictions_class_Test = (predictions_test[:,1] >= th).astype('int')

        report_performance(file,time,dataType,predictions_class_Train,predictions_class_Test, type_class,th, predictions_train[:,1], predictions_test[:,1],train_y,train_X.shape[0])

    # add new data
    
    return 0, 1

In [4]:
dtypes = {
    'comment_text':'str',
    'comment_id':'int',
    'page_id': 'str',
    'parent_id': 'str', 
    'post_id': 'str',
    'comment_reaction_count':'float',
    'comment_reply_count':'float',
    'comment_reaction1':'str',
    'comment_reaction2':'str',
    'comment_reaction3':'str',
    'comment_topfan':'float',
    'like':'float',
    'super_like':'float',
    'sup_like':'float',
    'angry':'float',
    'super_angry':'float',
    'sup_angry':'float',
    'love':'float',
    'super_love':'float',
    'sup_love':'float',
    'haha':'float',
    'super_haha':'float',
    'sup_haha':'float',
    'sad':'float',
    'super_sad':'float',
    'sup_sad':'float',
    'wow':'float',
    'super_wow':'float',
    'sup_wow':'float',
    'is_skoi':'float',
    'thread_level': 'float',
    'post_image': 'float',
    'post_video': 'float',
    'post_comment_count':'float',
    'post_share_count':'float',
    'post_reaction_count':'float',
    'post_like': 'float',
    'post_love': 'float',
    'post_haha': 'float', 
    'post_sad': 'float',
    'post_wow': 'float',
    'post_angry': 'float',
    'comment_image':'float',
    'comment_video':'float',
    'comment_year':'float',
    'comment_month': 'float',
    'comment_day': 'float',
    'comment_hour': 'float',
    'comment_minute': 'float',
    'comment_second': 'float',
    'comment_day_of_week': 'float',
    'post_year':'float',
    'post_month':'float',
    'post_day':'float',
    'post_hour': 'float',
    'post_minute': 'float',
    'post_second': 'float',
    'post_day_of_week': 'float',
    'comment_text_char_len': 'float',
    'comment_num_hashtag': 'float',
    'comment_avg_char_hashtag':'float',
    'comment_urls_len':'float',
    'comment_neg_emo_count':'float',
    'comment_neu_emo_count': 'float',
    'comment_pos_emo_count': 'float',
    'comment_author': 'str'
}
#,
context_features = [
    'comment_reaction_count','comment_reply_count', 'comment_reaction1','comment_reaction2','comment_reaction3',
    'comment_topfan','like','super_like','sup_like','angry','super_angry','sup_angry','love','super_love','sup_love',
    'haha','super_haha','sup_haha','sad','super_sad','sup_sad','wow','super_wow','sup_wow','is_skoi',
    'thread_level','post_image','post_video','post_comment_count','post_share_count','post_reaction_count',
    'post_like','post_love','post_haha','post_sad','post_wow','post_angry',
    'comment_image','comment_video','comment_year','comment_month','comment_day','comment_hour',
    'comment_minute','comment_second','comment_day_of_week','post_year','post_month',
    'post_day','post_hour','post_minute','post_second','post_day_of_week','comment_text_char_len',
    'comment_num_hashtag','comment_avg_char_hashtag','comment_urls_len','comment_neg_emo_count',
    'comment_neu_emo_count','comment_pos_emo_count']
#comment_author
content_features = ['comment_text']
labels = ['rude','figurative','offensive','dirty']


reaction_map = {
    'haha': 1,
    'like': 2,
    'angry': 3,
    'sad': 4,
    'wow': 5,
    'love': 6,
    '<NONE>': 7
}

In [5]:
import pandas as pd
import numpy as np
from pythainlp import word_tokenize
from tqdm import tqdm_notebook
import re
import emoji

#viz
from plotnine import *
import matplotlib.pyplot as plt
import seaborn as sns

from ast import literal_eval
from collections import Counter

#fastai
import fastai
from fastai.text import *
from fastai.callbacks import CSVLogger

#pythainlp
from pythainlp.ulmfit import *


import numpy as np
from sklearn.model_selection import train_test_split

from fastai.text import *
from fastai.callbacks import CSVLogger, SaveModelCallback
from pythainlp.ulmfit import *


from sklearn.metrics import f1_score

import time



from sklearn.metrics import confusion_matrix

In [6]:
def replace_url(text):
    URL_PATTERN = r"""(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}\[\]]+|\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\))+(?:\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))"""
    return re.sub(URL_PATTERN, 'xxurl', text)

def replace_rep(text):
    def _replace_rep(m):
        c,cc = m.groups()
        return f'{c}xxrep'
    re_rep = re.compile(r'(\S)(\1{2,})')
    return re_rep.sub(_replace_rep, text)

def ungroup_emoji(toks):
    res = []
    for tok in toks:
        if emoji.emoji_count(tok) == len(tok):
            for char in tok:
                res.append(char)
        else:
            res.append(tok)
    return res

def process_text(text):
    #pre rules
    res = text.lower().strip()
    res = replace_url(res)
    res = replace_rep(res)
    
    #tokenize
    res = [word for word in word_tokenize(res, engine='ulmfit') if word and not re.search(pattern=r"\s+", string=word)]
    
    #post rules
    res = ungroup_emoji(res)
    
    return res

In [7]:
res1 = np.load("embeding_arr.npy")
emb_dict = {v:i for i, v in enumerate(res1[:, 0])}
res1 = res1[:,1:]

def embedding_df(df):
    for i in range(400):
        df['emb_' + str(i)] = None

    a = np.zeros((len(df),400))
    for i, v in enumerate(df.comment_id):
        a[i] = res1[emb_dict[v]]

    df.iloc[:,df.columns.get_loc('emb_0'):] = a
    return df

In [8]:
def pre_df(df3):
    df3.rude = df3.rude.apply(int)
    df3.figurative = df3.figurative.apply(int)
    df3.offensive = df3.offensive.apply(int)
    df3.dirty = df3.dirty.apply(int)
    return df3

In [9]:
Corpus = pd.read_csv('../../MapLabeledData/UpdateLabel/dirty_vs_non.csv', dtype=dtypes)

In [10]:
reaction_columns = [f'comment_reaction{i}' for i in range(1, 4)]
Corpus[reaction_columns] = Corpus[reaction_columns].replace(reaction_map)

In [11]:
Corpus = pre_df(Corpus)
Train_X, Test_X = train_test_split(Corpus, train_size = 0.8)
Train_Y = Train_X[['rude','figurative','offensive','dirty']]
Test_Y = Test_X[['rude','figurative','offensive','dirty']]

In [12]:
df_embed_train = embedding_df(Train_X)
df_embed_test = embedding_df(Test_X)
embedding_train=df_embed_train.iloc[:, df_embed_train.columns.get_loc('emb_0'):]
embedding_test=df_embed_test.iloc[:, df_embed_test.columns.get_loc('emb_0'):]
    
train_embed_content = embedding_train
test_embed_content = embedding_test
    
Tfidf_vect = TfidfVectorizer(tokenizer=split_word, min_df=2, max_features=10000, token_pattern=None)
Tfidf_vect.fit(Train_X['comment_text'])
    
Train_X_Tfidf = Tfidf_vect.transform(Train_X['comment_text'])
Test_X_Tfidf = Tfidf_vect.transform(Test_X['comment_text'])
    
train_tfidf_content = Train_X_Tfidf
test_tfidf_content = Test_X_Tfidf

/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [15]:
from scipy.sparse import hstack, csr_matrix, vstack
file = '80_20_dirty_tfidf_embedding_svm.csv'
createGeneralEvaluation(file)
label = 'dirty'

train_feat_content = hstack([train_embed_content, train_tfidf_content])
test_feat_content = hstack([test_embed_content, test_tfidf_content])
    
_train_y = Train_Y[label].values
_test_y = Test_Y[label].values
content_train_y = _train_y

_cp, _cn = train_and_label(
train_feat_content, content_train_y, 
test_feat_content, _test_y,
file, label, 0, "content",'svm')

Test data of content
dirty Accuracy score: 16.373142542652726
Confusion Matrix: 
 [[ 473 3036]
 [   3  122]]
PRF: 
               precision    recall  f1-score   support

           0       0.99      0.13      0.24      3509
           1       0.04      0.98      0.07       125

    accuracy                           0.16      3634
   macro avg       0.52      0.56      0.16      3634
weighted avg       0.96      0.16      0.23      3634

Area under the curve ROC:  0.9048914220575662
Area under the curve PR:  0.4944805604482744
-----------------------------------------------------------

Train data of content
dirty Accuracy score: 16.27794977640179
Confusion Matrix: 
 [[ 1896 12169]
 [    0   470]]
PRF: 
               precision    recall  f1-score   support

           0       1.00      0.13      0.24     14065
           1       0.04      1.00      0.07       470

    accuracy                           0.16     14535
   macro avg       0.52      0.57      0.15     14535
weighted avg 

PRF: 
               precision    recall  f1-score   support

           0       0.97      1.00      0.98      3509
           1       0.00      0.00      0.00       125

    accuracy                           0.97      3634
   macro avg       0.48      0.50      0.49      3634
weighted avg       0.93      0.97      0.95      3634

Area under the curve ROC:  0.9048914220575662
Area under the curve PR:  0.4944805604482744
-----------------------------------------------------------

Train data of content
dirty Accuracy score: 96.76642586859305
Confusion Matrix: 
 [[14065     0]
 [  470     0]]
PRF: 
               precision    recall  f1-score   support

           0       0.97      1.00      0.98     14065
           1       0.00      0.00      0.00       470

    accuracy                           0.97     14535
   macro avg       0.48      0.50      0.49     14535
weighted avg       0.94      0.97      0.95     14535

Area under the curve ROC:  0.9995448185098063
Area under the curve 

/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

In [16]:
from scipy.sparse import hstack, csr_matrix, vstack
file = '80_20_combined_dirty_tfidf_embedding_svm.csv'
createGeneralEvaluation(file)
label = 'dirty'

train_feat_context = Train_X[context_features].values
test_feat_context = Test_X[context_features].values
    
train_feat = hstack([train_embed_content, train_tfidf_content, train_feat_context])
test_feat = hstack([test_embed_content, test_tfidf_content, test_feat_context])
    
_train_y = Train_Y[label].values
_test_y = Test_Y[label].values
content_train_y = _train_y

_cp, _cn = train_and_label(
train_feat, content_train_y, 
test_feat, _test_y,
file, label, 0, "combined","svm")

/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Test data of combined
dirty Accuracy score: 47.44083654375344
Confusion Matrix: 
 [[1601 1908]
 [   2  123]]
PRF: 
               precision    recall  f1-score   support

           0       1.00      0.46      0.63      3509
           1       0.06      0.98      0.11       125

    accuracy                           0.47      3634
   macro avg       0.53      0.72      0.37      3634
weighted avg       0.97      0.47      0.61      3634

Area under the curve ROC:  0.8788760330578512
Area under the curve PR:  0.13075381224114407
-----------------------------------------------------------

Train data of combined
dirty Accuracy score: 48.421052631578945
Confusion Matrix: 
 [[6580 7485]
 [  12  458]]
PRF: 
               precision    recall  f1-score   support

           0       1.00      0.47      0.64     14065
           1       0.06      0.97      0.11       470

    accuracy                           0.48     14535
   macro avg       0.53      0.72      0.37     14535
weighted avg  

Confusion Matrix: 
 [[14028    37]
 [  468     2]]
PRF: 
               precision    recall  f1-score   support

           0       0.97      1.00      0.98     14065
           1       0.05      0.00      0.01       470

    accuracy                           0.97     14535
   macro avg       0.51      0.50      0.50     14535
weighted avg       0.94      0.97      0.95     14535

Area under the curve ROC:  0.8734351907178678
Area under the curve PR:  0.1295021651804462
___________________________________________________________________________________________________________

Test data of combined
dirty Accuracy score: 96.56026417171161
Confusion Matrix: 
 [[3509    0]
 [ 125    0]]
PRF: 
               precision    recall  f1-score   support

           0       0.97      1.00      0.98      3509
           1       0.00      0.00      0.00       125

    accuracy                           0.97      3634
   macro avg       0.48      0.50      0.49      3634
weighted avg       0.93    

/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r